## Setup

In [28]:
# import arc libraries
from arcgis.gis import GIS
from arcgis.mapping import WebMap

import pandas as pd



In [29]:

# import and authenticate gee library
import ee
ee.Authenticate()
ee.Initialize()




Successfully saved authorization token.


## GEE

In [5]:
# get NAIP and TIGER catalogs
NAIP = ee.ImageCollection("USDA/NAIP/DOQQ")
TIGER = ee.FeatureCollection("TIGER/2018/States")


In [18]:
NAIP_dates = r"C:\Users\jczawlytko\code\LCCAA\NAIP_dates.csv"
df = pd.read_csv(NAIP_dates)
print(df.columns)

Index(['State', 'ST', 'STATEFP', 'T1', 'T2', 'T3'], dtype='object')


In [30]:
naip_viz_params = {
    'bands':['R', 'G', 'B'],
    'min': 0,
    'max': 200
    }


In [67]:
#reset tiles list
tiles = []

# loop thru df

for index, row in df.iterrows():
    # get state name and T1 and T2 NAIP years
    State = row ['State']
    T1 = row['T1']
    T2 = row['T2']
    print(f"Generating tiles for {row ['State']}                ", end='\r')
    
    # get stat bounds
    st_bounds = TIGER.filterMetadata('NAME', 'equals', State)

    #create state naip collection then filter by T1 and T2 dates
    st_naip = NAIP.filterBounds(st_bounds)
    naipT1 = st_naip.filterDate(f'{T1}-01-01', f'{T1}-12-31').median()
    naipT2 = st_naip.filterDate(f'{T2}-01-01', f'{T2}-12-31').median()

    #create 
    map_id_dict1 = ee.Image(naipT1).getMapId(naip_viz_params)
    map_id_dict2 = ee.Image(naipT2).getMapId(naip_viz_params)
    tiles1 = map_id_dict1['tile_fetcher'].url_format
    tiles2 = map_id_dict2['tile_fetcher'].url_format
    tiles.append(tiles1)
    tiles.append(tiles2)
print(f"{len(tiles)} sets of tiles generated                        ")

14 sets of tiles generated                        


## ESRI webmap

In [33]:
# first we create a arconline client with our credentials
gis = GIS(username =  "mevans_CC", password = "zpbGaefAHAYQ5H1VtXjt")

In [34]:
# find the instance of the webmap we want to display gee imagery on
search_results = gis.content.search(query = 'title:NAIP_viewer', item_type = 'Web Map')
wm_item = search_results[0]
web_map_obj = WebMap(wm_item)

In [62]:
layers = web_map_obj.layers
naip_layers = [layer for layer in layers if 'NAIP' in layer['title']]


In [57]:
[naip_layers[i].update({'templateUrl':tile}) for i, tile in enumerate(tiles)]

IndexError: list index out of range

In [40]:
web_map_obj.update()

True